In [156]:
import pathlib
import pandas as pd
import re
import numpy as np
from collections import Counter
from fuzzywuzzy import fuzz

import warnings
warnings.filterwarnings('ignore')

## Read data and extract table company - true_utes 

In [157]:
# Read xlsx from Enrique
df_checks = pd.read_excel("../data/ejemplos_enrique.xlsx")
df_checks

,Unnamed: 0,Nº EXPEDIENTE,TÍTULO DEL CONTRATO,EMPRESAS PARTICIPANTES RAZÓN SOCIAL,NIF,EMPRESAS QUE LICITAN EN UTE,NIF COMPONENTES UTE,PYME,ENLACE AL DOCUMENTO ACTA APERTURA SOBRE A,ACTA SOBRE A IDENTIFICA UTE,TIPO DE ENTIDAD,TIPO DE SOCIEDAD,EMPRESA ADJUDICATARIA,ENLACE AL DOCUMENTO ANUNCIO FORMALIZACIÓN
0,1.0,300/2021/00680 LOTE 1,PRESTACIÓN DE SERVICIOS PARA LA PUESTA EN MARC...,GLOBAL INCUBFATOR SL,B85432037,NaN,NaN,SI,https://contrataciondelestado.es/wps/wcm/conne...,NaN,Empresa,SL,NO,DOC_FORM2023-018297.pdf (contrataciondelestado...
1,NaN,NaN,NaN,"UTE TYPSA, ESTADÍSTICA Y SERVICIOS SLU Y MAGIC...",TEMP 09454,TÉCNICA Y PROYECTOS SA (TYPSA),A28171288,NO,NaN,"SÍ (Pág 3 ""(…) UTE TYPSA ESTADÍSTICA Y SERVICI...",Empresa,SLU,NO,NaN
2,NaN,NaN,NaN,NaN,NaN,MAGIC FENNEC SL,B99550667,SI,NaN,NaN,Empresa,SL,NaN,NaN
3,NaN,NaN,NaN,"UTE VODAFONE ESPAÑA, SAU",U72479678,VODAFONE ESPAÑA SAU,A80907397,NO,NaN,"SÍ (Pág 3 ""(…) UTE GAMERA, VODAFONE OWW GAMERS...",Empresa,SAU,SI,NaN
4,NaN,NaN,NaN,NaN,NaN,GAMERA NEST SL,B2882956K,SI,NaN,NaN,Empresa,SL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,NaN,NaN,NaN,"SERLINGO SOCIAL, S.L.U.",B86868593,NaN,NaN,SÍ,NaN,NaN,Empresa,Centro Especial de Empleo,NO,NaN
99,NaN,NaN,NaN,"TREBOL INTEGRACION SOCIAL, S.L.",B82349457,NaN,NaN,SÍ,NaN,NaN,Empresa,Centro Especial de Empleo,NO,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,18.0,300/2021/00569,“DISEÑO Y PUESTA EN MARCHA DE UN\nLABORATORIO ...,FUNDACION JUAN XXIII-RONCALLI PARA LA DISCAPAC...,G78280880,NaN,NaN,NO,https://contrataciondelestado.es/wps/wcm/conne...,NaN,Fundación,NaN,SÍ,https://contrataciondelestado.es/wps/wcm/conne...


In [158]:
# Get compnay and utes
df_checks = df_checks.rename(columns={
    "EMPRESAS PARTICIPANTES RAZÓN SOCIAL": "utes",
    "EMPRESAS QUE LICITAN EN UTE": "company"
})[["utes", "company"]]
df_checks

,utes,company
0,GLOBAL INCUBFATOR SL,NaN
1,"UTE TYPSA, ESTADÍSTICA Y SERVICIOS SLU Y MAGIC...",TÉCNICA Y PROYECTOS SA (TYPSA)
2,NaN,MAGIC FENNEC SL
3,"UTE VODAFONE ESPAÑA, SAU",VODAFONE ESPAÑA SAU
4,NaN,GAMERA NEST SL
...,...,...
98,"SERLINGO SOCIAL, S.L.U.",NaN
99,"TREBOL INTEGRACION SOCIAL, S.L.",NaN
100,NaN,NaN
101,FUNDACION JUAN XXIII-RONCALLI PARA LA DISCAPAC...,NaN


In [159]:
# Get name of utes that went to nan after convesion from excel
for index, row in df_checks.iterrows():
    if pd.isna(row["utes"]):
        row["utes"] = df_checks.iloc[index-1].utes
        
# We do not consider ute/compnay pairs with nan value in compnay
df_checks = df_checks.dropna()

# Save utes in a list for processing with our methids
utes_found = df_checks.utes.unique().tolist()

# Create a new dataframe where for each company, we have a list with the utes in which it has participated
new_df_checks = df_checks.groupby('company')['utes'].apply(list).reset_index()
new_df_checks.columns = ['company', 'true_utes']

# Extract list of companies
companies_list = new_df_checks.company.values.tolist()

new_df_checks

,company,true_utes
0,ADD4U SOLUCIONES PARA EL DESARROLLO S.L.,[UTE HOLISTIC - LIGHTHOUSE - ADD4U]
1,ADD4U SOLUCIONES PARA GESTION Y DESARROLLO SL.,[UTE ADD4U - SIFDI]
2,ALTRAPO LAB S COOP MAD,[UTE PIC FUENCARRAL]
3,ARACAS DE MANTENIMIENTO INTEGRAL S.A.,[UTE ARACAS DE MANTENIMIENTO INTEGRAL S.A - AR...
4,"ARACAS MANTENIMIENTO Y SERVICIO, S.L.",[UTE ARACAS DE MANTENIMIENTO INTEGRAL S.A - AR...
5,BEL CONSULTORES S.L.,[UTE LICITACIONES BEL CONSULTORES S.L. - FUNDA...
6,"CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO, S.A",[UTE CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO...
7,CROWE ACCELERA MANAGEMENT,[UTE UNIVERSIDAD REY JUAN CARLOS - CROWE ACEL...
8,FERNÁNDEZ MOLINA OBRAS Y SERVICIOS S.A.,[UTE CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO...
9,FUNDACION DELEGACIÓN FUNDACIÓN FINNOVA,[UTE FUNDACION DELEGACIÓN FUNDACIÓN FINNOVA - ...


## Get lists of stops

In [160]:
df_company = pd.read_parquet("../data/company_info.parquet")
SEPS_UTES = (df_company.CompanyType.unique().tolist() +
             ["s.l.p.", "s.l.p", "s.l.l.", "s.a.u.", "s.l.u.", "s.l.u", "s.l.u,", "slu", "s.l", "c.o.o.p.", "s.a", "sl.", "sccl", "s.coop.pequeña"] +
             ["sl", "slu", "s.", "scclp"])[1:]

OTHERS = ["-", "_", ",", "+", ".", ".,"]
SEPS = SEPS_UTES + OTHERS

In [161]:
path_aux = pathlib.Path("metalists")

In [162]:
########################################################################
# Create lists of common names/last names in Spain and broadcast them
########################################################################
path_apellidos = path_aux / "apellidos.parquet"
path_nombres = path_aux / "nombres.parquet"

# Read Parquet files into Pandas DataFrames
df_appellidos = pd.read_parquet(path_apellidos)
df_nombres = pd.read_parquet(path_nombres)

# Apply transformation to divide composed names into new items
df_appellidos['palabras'] = df_appellidos['apellidos'].str.split(" ")
df_appellidos = df_appellidos.explode('palabras').rename(columns={'palabras': 'apellidos'})

df_nombres['palabras'] = df_nombres['nombres'].str.split(" ")
df_nombres = df_nombres.explode('palabras').rename(columns={'palabras': 'nombres'})

# Collect the unique names and last names
unique_apellidos = df_appellidos['apellidos'].values.tolist()
unique_names = df_nombres['nombres'].values.tolist()
UNIQUE_NAMES_APELLIDOS = np.unique(np.concatenate((unique_names, unique_apellidos))).tolist()
UNIQUE_NAMES_APELLIDOS.sort()

In [163]:
########################################################################
# Create list of common spanish and catalan words
########################################################################
with open(path_aux.joinpath("es.txt")) as file:
    COMMON_SPANISH = file.readlines()
    COMMON_SPANISH = [line.rstrip() for line in COMMON_SPANISH]
with open(path_aux.joinpath("catalan.txt")) as file:
    COMMON_CATALAN = file.readlines()
    COMMON_CATALAN = [line.rstrip() for line in COMMON_CATALAN]

########################################################################
# We also laod a list of words generated from the "splits" of the company names obtained from a previous execution of this script that have a frequency larger than the 95th percentile of the frequency distribution of the "splits" of the company names
########################################################################
with open(path_aux.joinpath("filtered_elements.txt")) as file:
    SPLITS_STOPS = file.readlines()
    SPLITS_STOPS = [line.rstrip() for line in SPLITS_STOPS]

In [164]:
# GLOBAL VARIABLES
re_pattern = re.compile(r'([.,!?;:\s()-])')
TRANS_TABLE = str.maketrans('áéíóúÁÉÍÓÚòÒ', 'aeiouAEIOUoO')
def remove_punctuation(
    name: str
) -> str:
    """Remove punctuation and accents from a string.

    Parameters
    ----------
    name : str
        The name of the company.

    Returns
    -------
    str
        The name of the company without punctuation and accents.
    """
    return name.translate(TRANS_TABLE)

def remove_punctuation_list(
    name: str
) -> str:
    return [remove_punctuation(el) for el in name]

def find_substring_similar(
    main_string: str,
    substring: str,
    substring_splits: list,
    threshold: int = 95
) -> bool:
    """
    Given the name of an UTE (main_string), that of a company (substring), and the "splits" of the company name (i.e. a list of the words that compose the company name, after filtering out some terms), it first try to find the substring within the main_string with a similarity ratio larger than a threshold. If it does not find it, it checks for partial matches with some punctuation separation.

    Parameters
    ----------
    main_string : str
        The name of the UTE.
    substring : str
        The name of the company.
    substring_splits : list
        The "splits" of the company name, after filtering out some terms.
    threshold : int
        The threshold for the similarity ratio. The default is 95.

    Returns
    -------
    bool
        Whether the substring is found within the main_string with a similarity ratio larger than a threshold or not.
    """

    ############################################################################
    # Check for exact matches
    ############################################################################
    regex_pattern = r'\b' + re.escape(substring) + r'\b'
    regex = re.compile(regex_pattern)

    len_substring = len(substring)
    for match in regex.finditer(main_string):
        start_index = match.start()
        end_index = match.end()

        # Calculate similarity only for substrings around the match
        for i in range(start_index, end_index - len_substring + 1):
            sub = main_string[i:i + len_substring]
            similitud = fuzz.ratio(sub, substring)
            if similitud >= threshold:
                return True

    ############################################################################
    # Check for partial matches with some punctuation separation
    ############################################################################
    if re.search(r'[.,!?;:\s()-]', main_string):
        main_string_split = set(
            re.split(r'([.,!?;:\s()-])', main_string.lower()))
        # if any(len(split) > 3 and split in main_string_split for split in substring_splits):
        substring_set = set(substring_splits)

        if len(main_string_split.intersection(substring_set)) >= 1:
            return True
        
def get_splits(
    name: str
) -> list:
    """Split the name of a company into its parts, after filtering out some terms.

    The terms that are filtered out are:
    - Words with less than 3 characters.
    - Numbers.
    - Common words in Spanish and Catalan (extracted from the files "es.txt" and "catalan.txt").
    - Common last names and names in Spain (extracted from the files "apellidos.parquet" and "nombres.parquet").
    - Common words obtained from the "splits" of the company names that have a frequency larger than the 95th percentile of the frequency distribution of the "splits" of the company names (extracted from the file "filtered_elements.txt").

    Parameters
    ----------
    name : str
        The name of the company.

    Returns
    -------
    list
        The parts of the name of the company, after filtering out some terms.
    """
    if re_pattern.search(name):
        name_parts = re_pattern.split(name.lower())
        # Remove empty strings, separators, and duplicates
        name_parts = [part.strip(',').strip()
                      for part in name_parts
                      if part.strip()
                      and len(part) > 3
                      and not part.isdigit()
                      and not part in SEPS
                      and not part in (UNIQUE_NAMES_APELLIDOS + COMMON_SPANISH + COMMON_CATALAN + SPLITS_STOPS)
                     ]
        return name_parts if len(name_parts) > 0 else []
    return []

def get_company_utes(name, splits):
    aux = [fullName for fullName in unique_names if find_substring_similar(fullName, name, splits)]
    
    return list(set(aux))

In [165]:
new_df_checks["company_norm"] = new_df_checks["company"].apply(remove_punctuation)
new_df_checks["true_utes_norm"] = new_df_checks["true_utes"].apply(remove_punctuation_list)
new_df_checks["splits"] = new_df_checks["company_norm"].apply(get_splits)

In [166]:
path_utes = pathlib.Path("../data/utes.parquet")
df_utes = pd.read_parquet(path_utes)
df_utes["FullName_norm"] = df_utes["FullName"].apply(remove_punctuation)
#unique_names = df_utes.FullName_norm.unique().tolist()

def flatten(xss):
    return [x for xs in xss for x in xs]
unique_names = flatten(new_df_checks.true_utes_norm.values.tolist())

In [167]:
new_df_checks['utes'] = new_df_checks.apply(
    lambda row: get_company_utes(row['company_norm'], row["splits"]), axis=1)

In [168]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                      'display.max_colwidth',None):
    display(new_df_checks)

,company,true_utes,company_norm,true_utes_norm,splits,utes
0,ADD4U SOLUCIONES PARA EL DESARROLLO S.L.,[UTE HOLISTIC - LIGHTHOUSE - ADD4U],ADD4U SOLUCIONES PARA EL DESARROLLO S.L.,[UTE HOLISTIC - LIGHTHOUSE - ADD4U],[add4u],"[UTE ADD4U - SIFDI, UTE HOLISTIC - LIGHTHOUSE - ADD4U]"
1,ADD4U SOLUCIONES PARA GESTION Y DESARROLLO SL.,[UTE ADD4U - SIFDI],ADD4U SOLUCIONES PARA GESTION Y DESARROLLO SL.,[UTE ADD4U - SIFDI],[add4u],"[UTE ADD4U - SIFDI, UTE HOLISTIC - LIGHTHOUSE - ADD4U]"
2,ALTRAPO LAB S COOP MAD,[UTE PIC FUENCARRAL],ALTRAPO LAB S COOP MAD,[UTE PIC FUENCARRAL],"[altrapo, coop]",[]
3,ARACAS DE MANTENIMIENTO INTEGRAL S.A.,[UTE ARACAS DE MANTENIMIENTO INTEGRAL S.A - ARACAS MANTENIMIENTO Y SERVICIO S.L.],ARACAS DE MANTENIMIENTO INTEGRAL S.A.,[UTE ARACAS DE MANTENIMIENTO INTEGRAL S.A - ARACAS MANTENIMIENTO Y SERVICIO S.L.],[aracas],[UTE ARACAS DE MANTENIMIENTO INTEGRAL S.A - ARACAS MANTENIMIENTO Y SERVICIO S.L.]
4,"ARACAS MANTENIMIENTO Y SERVICIO, S.L.",[UTE ARACAS DE MANTENIMIENTO INTEGRAL S.A - ARACAS MANTENIMIENTO Y SERVICIO S.L.],"ARACAS MANTENIMIENTO Y SERVICIO, S.L.",[UTE ARACAS DE MANTENIMIENTO INTEGRAL S.A - ARACAS MANTENIMIENTO Y SERVICIO S.L.],[aracas],[UTE ARACAS DE MANTENIMIENTO INTEGRAL S.A - ARACAS MANTENIMIENTO Y SERVICIO S.L.]
5,BEL CONSULTORES S.L.,[UTE LICITACIONES BEL CONSULTORES S.L. - FUNDACIÓN CIDEAL - SECTOR 3 INFORMACIÓN Y SERVICIO - GESOR],BEL CONSULTORES S.L.,[UTE LICITACIONES BEL CONSULTORES S.L. - FUNDACION CIDEAL - SECTOR 3 INFORMACION Y SERVICIO - GESOR],[],[]
6,"CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO, S.A","[UTE CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO, S.A. - OBRAS Y SERVICIOS F. MOLINA]","CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO, S.A","[UTE CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO, S.A. - OBRAS Y SERVICIOS F. MOLINA]",[],[]
7,CROWE ACCELERA MANAGEMENT,[UTE UNIVERSIDAD REY JUAN CARLOS - CROWE ACELERA MANAGEMENT],CROWE ACCELERA MANAGEMENT,[UTE UNIVERSIDAD REY JUAN CARLOS - CROWE ACELERA MANAGEMENT],[accelera],[]
8,FERNÁNDEZ MOLINA OBRAS Y SERVICIOS S.A.,"[UTE CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO, S.A. - OBRAS Y SERVICIOS F. MOLINA]",FERNANDEZ MOLINA OBRAS Y SERVICIOS S.A.,"[UTE CONSTRUCCIONES FRANCISCO CARRASCO NOVILLO, S.A. - OBRAS Y SERVICIOS F. MOLINA]",[],[]
9,FUNDACION DELEGACIÓN FUNDACIÓN FINNOVA,[UTE FUNDACION DELEGACIÓN FUNDACIÓN FINNOVA - RECZYCLIA],FUNDACION DELEGACION FUNDACION FINNOVA,[UTE FUNDACION DELEGACION FUNDACION FINNOVA - RECZYCLIA],[finnova],[UTE FUNDACION DELEGACION FUNDACION FINNOVA - RECZYCLIA]
